In [3]:

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from skimage.io import imsave
import os
import shutil

img_height = 28
img_width = 28
img_size = img_height * img_width

to_train = True
to_restore = False
output_path = "output"

max_epoch = 500

h1_size = 150
h2_size = 300
z_size = 100
batch_size = 256

# 일반적인 GAN 의 형태 입니다.
# 라벨을 구분하지 않습니다.

# 제너레이터 (G)
def build_generator(z_prior):
    # Fully Connected Layer 1 (100 (latent-vector) -> 150 (h1_size))
    w1 = tf.Variable(tf.truncated_normal([z_size, h1_size], stddev=0.1), name="g_w1", dtype=tf.float32)
    b1 = tf.Variable(tf.zeros([h1_size]), name="g_b1", dtype=tf.float32)
    h1 = tf.nn.relu(tf.matmul(z_prior, w1) + b1)

    # Fully Connected Layer 2 (150 (h1_size) -> 300 (h2_size))
    w2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=0.1), name="g_w2", dtype=tf.float32)
    b2 = tf.Variable(tf.zeros([h2_size]), name="g_b2", dtype=tf.float32)
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)

    # Fully Connected Layer 3 (300 (h1_size) -> input_height * input_width (img_size))
    w3 = tf.Variable(tf.truncated_normal([h2_size, img_size], stddev=0.1), name="g_w3", dtype=tf.float32)
    b3 = tf.Variable(tf.zeros([img_size]), name="g_b3", dtype=tf.float32)
    h3 = tf.matmul(h2, w3) + b3
    # 마지막은 활성화함수 tanh 로
    x_generate = tf.nn.tanh(h3)

    # generator 변수 저장
    g_params = [w1, b1, w2, b2, w3, b3]

    return x_generate, g_params

# 디스크리미네이터 (D)
def build_discriminator(x_data, x_generated, keep_prob):
    # 실제 이미지와 생성된 이미지를 합침
    x_in = tf.concat([x_data, x_generated], 0) 

    # Fully Connected Layer 1 (input_height * input_width (img_size) -> 200 (h2_size)) , dropout
    w1 = tf.Variable(tf.truncated_normal([img_size, h2_size], stddev=0.1), name="d_w1", dtype=tf.float32)
    b1 = tf.Variable(tf.zeros([h2_size]), name="d_b1", dtype=tf.float32)
    h1 = tf.nn.dropout(tf.nn.relu(tf.matmul(x_in, w1) + b1), keep_prob)

    # Fully Connected Layer 2 (200 (h1_size) -> 150 (h1_size)) , dropout
    w2 = tf.Variable(tf.truncated_normal([h2_size, h1_size], stddev=0.1), name="d_w2", dtype=tf.float32)
    b2 = tf.Variable(tf.zeros([h1_size]), name="d_b2", dtype=tf.float32)
    h2 = tf.nn.dropout(tf.nn.relu(tf.matmul(h1, w2) + b2), keep_prob)

    # Fully Connected Layer 3 (150 (h1_size) -> 1)
    w3 = tf.Variable(tf.truncated_normal([h1_size, 1], stddev=0.1), name="d_w3", dtype=tf.float32)
    b3 = tf.Variable(tf.zeros([1]), name="d_b3", dtype=tf.float32)
    h3 = tf.matmul(h2, w3) + b3

    # batch_size 만큼 잘라 각각 y_data, y_generated 로
    # ex)
    #   이미지 60000개, 배치 사이즈 256, 이미지 사이즈 28 * 28 이라면
    #   h3 shape : (257, 1)
    #   y_data shape : (256, 1)
    #   y_generated shape : (1, 1) 
    y_data = tf.nn.sigmoid(tf.slice(h3, [0, 0], [batch_size, -1], name=None))
    y_generated = tf.nn.sigmoid(tf.slice(h3, [batch_size, 0], [-1, -1], name=None))

    # discriminator 변수 저장
    d_params = [w1, b1, w2, b2, w3, b3]

    return y_data, y_generated, d_params

# 결과 저장 (이미지)
def show_result(batch_res, fname, grid_size=(8, 8), grid_pad=5):
    batch_res = 0.5 * batch_res.reshape((batch_res.shape[0], img_height, img_width)) + 0.5
    img_h, img_w = batch_res.shape[1], batch_res.shape[2]
    grid_h = img_h * grid_size[0] + grid_pad * (grid_size[0] - 1)
    grid_w = img_w * grid_size[1] + grid_pad * (grid_size[1] - 1)
    img_grid = np.zeros((grid_h, grid_w), dtype=np.uint8)
    for i, res in enumerate(batch_res):
        if i >= grid_size[0] * grid_size[1]:
            break
        img = (res) * 255.
        img = img.astype(np.uint8)
        row = (i // grid_size[0]) * (img_h + grid_pad)
        col = (i % grid_size[1]) * (img_w + grid_pad)
        img_grid[row:row + img_h, col:col + img_w] = img
    imsave(fname, img_grid)


def train():
    # mnist 로 학습할 경우
    # mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    
    # phd08 로 학습할 경우
    phd08 = np.load('exampledata/phd08_data_1.npy')
    size = phd08.shape[0]
    phd08 = phd08.reshape((size,784))

    x_data = tf.placeholder(tf.float32, [batch_size, img_size], name="x_data") # (batch_size, img_size)
    z_prior = tf.placeholder(tf.float32, [batch_size, z_size], name="z_prior") # (batch_size, z_size)
    keep_prob = tf.placeholder(tf.float32, name="keep_prob") # dropout 퍼센트
    global_step = tf.Variable(0, name="global_step", trainable=False)

    # x_generated : generator 가 생성한 이미지, g_params : generater 의 TF 변수들
    x_generated, g_params = build_generator(z_prior)
    # 실제이미지, generater 가 생성한 이미지, dropout keep_prob 를 넣고 discriminator(경찰) 이 감별
    y_data, y_generated, d_params = build_discriminator(x_data, x_generated, keep_prob)

    # loss 함수 ( D 와 G 를 따로 ) *
    d_loss = - (tf.log(y_data) + tf.log(1 - y_generated))
    g_loss = - tf.log(y_generated)

    # optimizer : AdamOptimizer 사용 *
    optimizer = tf.train.AdamOptimizer(0.0001)

    # discriminator 와 generator 의 변수로 각각의 loss 함수를 최소화시키도록 학습
    d_trainer = optimizer.minimize(d_loss, var_list=d_params)
    g_trainer = optimizer.minimize(g_loss, var_list=g_params)

    init = tf.global_variables_initializer()

    saver = tf.train.Saver()

    sess = tf.Session()

    sess.run(init)

    if to_restore:
        chkpt_fname = tf.train.latest_checkpoint(output_path)
        saver.restore(sess, chkpt_fname)
    else:
        if os.path.exists(output_path):
            shutil.rmtree(output_path)
        os.mkdir(output_path)

    z_sample_val = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)

    for i in range(sess.run(global_step), max_epoch):
        for j in range(21870 // batch_size):
            print("epoch:%s, iter:%s" % (i, j))

            # x_value, _ = mnist.train.next_batch(batch_size)
            # x_value = 2 * x_value.astype(np.float32) - 1
            # print(x_value[0])

            batch_end = j * batch_size + batch_size
            if batch_end >= size:
                batch_end = size - 1
            x_value = phd08[ j * batch_size : batch_end ]
            x_value = x_value / 255.
            x_value = 2 * x_value - 1

            z_value = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
            sess.run(d_trainer,
                     feed_dict={x_data: x_value, z_prior: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
            if j % 1 == 0:
                sess.run(g_trainer,
                         feed_dict={x_data: x_value, z_prior: z_value, keep_prob: np.sum(0.7).astype(np.float32)})
        x_gen_val = sess.run(x_generated, feed_dict={z_prior: z_sample_val})
        show_result(x_gen_val, os.path.join(output_path, "sample%s.jpg" % i))
        print(x_gen_val)
        z_random_sample_val = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
        x_gen_val = sess.run(x_generated, feed_dict={z_prior: z_random_sample_val})
        show_result(x_gen_val, os.path.join(output_path, "random_sample%s.jpg" % i))
        sess.run(tf.assign(global_step, i + 1))
        saver.save(sess, os.path.join(output_path, "model"), global_step=global_step)

# 학습 완료 후 테스트 (이미지로 저장)
def test():
    z_prior = tf.placeholder(tf.float32, [batch_size, z_size], name="z_prior")
    x_generated, _ = build_generator(z_prior)
    chkpt_fname = tf.train.latest_checkpoint(output_path)

    init = tf.global_variables_initializer()
    sess = tf.Session()
    saver = tf.train.Saver()
    sess.run(init)
    saver.restore(sess, chkpt_fname)
    z_test_value = np.random.normal(0, 1, size=(batch_size, z_size)).astype(np.float32)
    x_gen_val = sess.run(x_generated, feed_dict={z_prior: z_test_value})
    show_result(x_gen_val, os.path.join(output_path, "test_result.jpg"))


if __name__ == '__main__':
    if to_train:
        train()
    else:
        test()


epoch:0, iter:0
epoch:0, iter:1
epoch:0, iter:2
epoch:0, iter:3
epoch:0, iter:4
epoch:0, iter:5
epoch:0, iter:6
epoch:0, iter:7
epoch:0, iter:8
epoch:0, iter:9
epoch:0, iter:10
epoch:0, iter:11
epoch:0, iter:12
epoch:0, iter:13
epoch:0, iter:14
epoch:0, iter:15
epoch:0, iter:16
epoch:0, iter:17
epoch:0, iter:18
epoch:0, iter:19
epoch:0, iter:20
epoch:0, iter:21
epoch:0, iter:22
epoch:0, iter:23
epoch:0, iter:24
epoch:0, iter:25
epoch:0, iter:26
epoch:0, iter:27
epoch:0, iter:28
epoch:0, iter:29
epoch:0, iter:30
epoch:0, iter:31
epoch:0, iter:32
epoch:0, iter:33
epoch:0, iter:34
epoch:0, iter:35
epoch:0, iter:36
epoch:0, iter:37
epoch:0, iter:38
epoch:0, iter:39
epoch:0, iter:40
epoch:0, iter:41
epoch:0, iter:42
epoch:0, iter:43
epoch:0, iter:44
epoch:0, iter:45
epoch:0, iter:46
epoch:0, iter:47
epoch:0, iter:48
epoch:0, iter:49
epoch:0, iter:50
epoch:0, iter:51
epoch:0, iter:52
epoch:0, iter:53
epoch:0, iter:54
epoch:0, iter:55
epoch:0, iter:56
epoch:0, iter:57
epoch:0, iter:58
epoch:0

epoch:4, iter:28
epoch:4, iter:29
epoch:4, iter:30
epoch:4, iter:31
epoch:4, iter:32
epoch:4, iter:33
epoch:4, iter:34
epoch:4, iter:35
epoch:4, iter:36
epoch:4, iter:37
epoch:4, iter:38
epoch:4, iter:39
epoch:4, iter:40
epoch:4, iter:41
epoch:4, iter:42
epoch:4, iter:43
epoch:4, iter:44
epoch:4, iter:45
epoch:4, iter:46
epoch:4, iter:47
epoch:4, iter:48
epoch:4, iter:49
epoch:4, iter:50
epoch:4, iter:51
epoch:4, iter:52
epoch:4, iter:53
epoch:4, iter:54
epoch:4, iter:55
epoch:4, iter:56
epoch:4, iter:57
epoch:4, iter:58
epoch:4, iter:59
epoch:4, iter:60
epoch:4, iter:61
epoch:4, iter:62
epoch:4, iter:63
epoch:4, iter:64
epoch:4, iter:65
epoch:4, iter:66
epoch:4, iter:67
epoch:4, iter:68
epoch:4, iter:69
epoch:4, iter:70
epoch:4, iter:71
epoch:4, iter:72
epoch:4, iter:73
epoch:4, iter:74
epoch:4, iter:75
epoch:4, iter:76
epoch:4, iter:77
epoch:4, iter:78
epoch:4, iter:79
epoch:4, iter:80
epoch:4, iter:81
epoch:4, iter:82
epoch:4, iter:83
epoch:4, iter:84
[[-0.79153562 -0.9895128  -0.95

epoch:8, iter:56
epoch:8, iter:57
epoch:8, iter:58
epoch:8, iter:59
epoch:8, iter:60
epoch:8, iter:61
epoch:8, iter:62
epoch:8, iter:63
epoch:8, iter:64
epoch:8, iter:65
epoch:8, iter:66
epoch:8, iter:67
epoch:8, iter:68
epoch:8, iter:69
epoch:8, iter:70
epoch:8, iter:71
epoch:8, iter:72
epoch:8, iter:73
epoch:8, iter:74
epoch:8, iter:75
epoch:8, iter:76
epoch:8, iter:77
epoch:8, iter:78
epoch:8, iter:79
epoch:8, iter:80
epoch:8, iter:81
epoch:8, iter:82
epoch:8, iter:83
epoch:8, iter:84
[[-0.94956118 -0.16705096  0.45264438 ..., -0.93641776 -0.96236306
  -0.25328761]
 [-0.98309928  0.88784099  0.76036417 ..., -0.98420286 -0.9640674
  -0.86285967]
 [-0.87785167  0.63025546  0.61854017 ..., -0.94597059 -0.93341476
  -0.04930953]
 ..., 
 [-0.76559412  0.76753402  0.69432718 ..., -0.87390399 -0.96991491
  -0.50307333]
 [-0.98542422  0.53938133  0.68338722 ..., -0.97594529 -0.91517919
  -0.70775008]
 [-0.99190652  0.65288609  0.6841526  ..., -0.7951594  -0.87668389
  -0.84393036]]
epoch:9,

epoch:12, iter:72
epoch:12, iter:73
epoch:12, iter:74
epoch:12, iter:75
epoch:12, iter:76
epoch:12, iter:77
epoch:12, iter:78
epoch:12, iter:79
epoch:12, iter:80
epoch:12, iter:81
epoch:12, iter:82
epoch:12, iter:83
epoch:12, iter:84
[[ 0.80186683 -0.10309166 -0.99424964 ..., -0.97515315 -0.80400282
  -0.79054219]
 [ 0.80404174  0.68977839 -0.98425567 ..., -0.97837782 -0.88894004
  -0.92808706]
 [ 0.94925547  0.72230786 -0.99466902 ..., -0.97334021 -0.53141975
  -0.58596551]
 ..., 
 [ 0.93463242  0.48751757 -0.92920798 ..., -0.88062215 -0.79953843
  -0.66076314]
 [ 0.74811178  0.45996425 -0.98296666 ..., -0.96765661 -0.78948843
  -0.87865222]
 [ 0.26692379  0.13956426 -0.92403442 ..., -0.7391361  -0.40686569
  -0.83653581]]
epoch:13, iter:0
epoch:13, iter:1
epoch:13, iter:2
epoch:13, iter:3
epoch:13, iter:4
epoch:13, iter:5
epoch:13, iter:6
epoch:13, iter:7
epoch:13, iter:8
epoch:13, iter:9
epoch:13, iter:10
epoch:13, iter:11
epoch:13, iter:12
epoch:13, iter:13
epoch:13, iter:14
epoch:

epoch:16, iter:79
epoch:16, iter:80
epoch:16, iter:81
epoch:16, iter:82
epoch:16, iter:83
epoch:16, iter:84
[[-0.60888636 -0.96680659 -0.98163122 ..., -0.61911237 -0.75512952
  -0.37411579]
 [-0.8690896  -0.87554413 -0.93833041 ..., -0.89264911 -0.94014961
  -0.6977939 ]
 [-0.20420381 -0.78267592 -0.97099996 ..., -0.604886   -0.57341766
  -0.19100924]
 ..., 
 [-0.30486616 -0.7690835  -0.86925787 ..., -0.43960035 -0.87682843
  -0.43147728]
 [-0.82956684 -0.88413322 -0.93863451 ..., -0.80883807 -0.85739434
  -0.59661043]
 [-0.88316596 -0.87973094 -0.91707098 ..., -0.14741644 -0.76614463
  -0.48332113]]
epoch:17, iter:0
epoch:17, iter:1
epoch:17, iter:2
epoch:17, iter:3
epoch:17, iter:4
epoch:17, iter:5
epoch:17, iter:6
epoch:17, iter:7
epoch:17, iter:8
epoch:17, iter:9
epoch:17, iter:10
epoch:17, iter:11
epoch:17, iter:12
epoch:17, iter:13
epoch:17, iter:14
epoch:17, iter:15
epoch:17, iter:16
epoch:17, iter:17
epoch:17, iter:18
epoch:17, iter:19
epoch:17, iter:20
epoch:17, iter:21
epoch:

...

epoch:98, iter:63
epoch:98, iter:64
epoch:98, iter:65
epoch:98, iter:66
epoch:98, iter:67
epoch:98, iter:68
epoch:98, iter:69
epoch:98, iter:70
epoch:98, iter:71
epoch:98, iter:72
epoch:98, iter:73
epoch:98, iter:74
epoch:98, iter:75
epoch:98, iter:76
epoch:98, iter:77
epoch:98, iter:78
epoch:98, iter:79
epoch:98, iter:80
epoch:98, iter:81
epoch:98, iter:82
epoch:98, iter:83
epoch:98, iter:84
[[ 0.75332332 -0.73331535 -0.61590725 ..., -0.64734703 -0.97353482
  -0.83257055]
 [-0.65353119 -0.99255651 -0.84134454 ..., -0.95300281 -0.99435663
  -0.94401759]
 [ 0.06560896 -0.70297277 -0.66299093 ..., -0.83065718 -0.95666295
  -0.93311369]
 ..., 
 [-0.08533923 -0.81916618 -0.77575004 ..., -0.73714167 -0.98285323
  -0.90969366]
 [ 0.74438339 -0.18313073 -0.45239872 ..., -0.637317   -0.91913152
  -0.48134869]
 [-0.72861075 -0.89837402 -0.85375839 ..., -0.83939493 -0.96437049
  -0.97750515]]
epoch:99, iter:0
epoch:99, iter:1
epoch:99, iter:2
epoch:99, iter:3
epoch:99, iter:4
epoch:99, iter:5
ep

epoch:102, iter:58
epoch:102, iter:59
epoch:102, iter:60
epoch:102, iter:61
epoch:102, iter:62
epoch:102, iter:63
epoch:102, iter:64
epoch:102, iter:65
epoch:102, iter:66
epoch:102, iter:67
epoch:102, iter:68
epoch:102, iter:69
epoch:102, iter:70
epoch:102, iter:71
epoch:102, iter:72
epoch:102, iter:73
epoch:102, iter:74
epoch:102, iter:75
epoch:102, iter:76
epoch:102, iter:77
epoch:102, iter:78
epoch:102, iter:79
epoch:102, iter:80
epoch:102, iter:81
epoch:102, iter:82
epoch:102, iter:83
epoch:102, iter:84
[[ 0.53561354 -0.58424824  0.50302994 ..., -0.66699183 -0.974958
  -0.92459017]
 [-0.97054946 -0.99764425  0.51482898 ..., -0.99067742 -0.99817789
  -0.98671871]
 [-0.28792191 -0.57281643  0.34525603 ..., -0.89496446 -0.97005266
  -0.96677542]
 ..., 
 [-0.60872567 -0.90790325 -0.14029965 ..., -0.91416615 -0.98329645
  -0.96181822]
 [ 0.48845187  0.03587046  0.47141385 ..., -0.76821518 -0.95423579
  -0.79578543]
 [-0.97478902 -0.94325984 -0.4051649  ..., -0.95830643 -0.85175812
  -0.

epoch:106, iter:48
epoch:106, iter:49
epoch:106, iter:50
epoch:106, iter:51
epoch:106, iter:52
epoch:106, iter:53
epoch:106, iter:54
epoch:106, iter:55
epoch:106, iter:56
epoch:106, iter:57
epoch:106, iter:58
epoch:106, iter:59
epoch:106, iter:60
epoch:106, iter:61
epoch:106, iter:62
epoch:106, iter:63
epoch:106, iter:64
epoch:106, iter:65
epoch:106, iter:66
epoch:106, iter:67
epoch:106, iter:68
epoch:106, iter:69
epoch:106, iter:70
epoch:106, iter:71
epoch:106, iter:72
epoch:106, iter:73
epoch:106, iter:74
epoch:106, iter:75
epoch:106, iter:76
epoch:106, iter:77
epoch:106, iter:78
epoch:106, iter:79
epoch:106, iter:80
epoch:106, iter:81
epoch:106, iter:82
epoch:106, iter:83
epoch:106, iter:84
[[ 0.34338325  0.42026931 -0.80714345 ...,  0.35615826 -0.97158152
  -0.91842866]
 [-0.99413502 -0.9870593  -0.94251972 ..., -0.9458546  -0.9985106
  -0.99739426]
 [-0.12719785  0.18007649 -0.68224728 ..., -0.34046924 -0.96160614
  -0.97012722]
 ..., 
 [-0.38829556 -0.5550037  -0.71395767 ...,  0

epoch:110, iter:37
epoch:110, iter:38
epoch:110, iter:39
epoch:110, iter:40
epoch:110, iter:41
epoch:110, iter:42
epoch:110, iter:43
epoch:110, iter:44
epoch:110, iter:45
epoch:110, iter:46
epoch:110, iter:47
epoch:110, iter:48
epoch:110, iter:49
epoch:110, iter:50
epoch:110, iter:51
epoch:110, iter:52
epoch:110, iter:53
epoch:110, iter:54
epoch:110, iter:55
epoch:110, iter:56
epoch:110, iter:57
epoch:110, iter:58
epoch:110, iter:59
epoch:110, iter:60
epoch:110, iter:61
epoch:110, iter:62
epoch:110, iter:63
epoch:110, iter:64
epoch:110, iter:65
epoch:110, iter:66
epoch:110, iter:67
epoch:110, iter:68
epoch:110, iter:69
epoch:110, iter:70
epoch:110, iter:71
epoch:110, iter:72
epoch:110, iter:73
epoch:110, iter:74
epoch:110, iter:75
epoch:110, iter:76
epoch:110, iter:77
epoch:110, iter:78
epoch:110, iter:79
epoch:110, iter:80
epoch:110, iter:81
epoch:110, iter:82
epoch:110, iter:83
epoch:110, iter:84
[[-0.28790939 -0.04364088 -0.26959413 ..., -0.82716572 -0.95662135
  -0.25580227]
 [-0.9

KeyboardInterrupt: 